In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import radians, sin, cos, asin, sqrt, pi, atan, atan2, fabs

In [8]:
def calcular_distancia(lat1,long1,lat2,long2):
    lon1, lat1 = (radians(coord) for coord in (long1,lat1))
    lon2, lat2 = (radians(coord) for coord in (long2,lat2))
    dlat = (lat2 - lat1)
    dlon = (lon2 - lon1)
    a = (
        sin(dlat * 0.5)**2 +
        cos(lat1) * cos(lat2) * sin(dlon * 0.5)**2
    )
    
    radioTierra = 6371008.8
    return 2 * radioTierra * asin(sqrt(a)) / 1000

#distancia de Haversine
#https://gist.github.com/habibutsu/8bbcc202a915e965c6a6d4f561d0e482

In [5]:
def distancia_minima(fila, dataframe):
    lat, lon = fila
    distancias = []
    for i in range (len(dataframe)):
        d = calcular_distancia(lat,lon,dataframe.lat[i], dataframe.lon[i])
        distancias.append(d)
    return min(distancias)

## Propiedades solo de Capital

In [ ]:
propiedades = pd.read_csv('properati.csv')
#propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/properati.csv')

In [ ]:
solo_capital = propiedades.loc[(propiedades.state_name == 'Capital Federal') & (propiedades.lat.notnull()) & (propiedades.lon.notnull()),:]
solo_capital = solo_capital.loc[((solo_capital.lat <= -34) & (solo_capital.lat >= -35)),:]
solo_capital = solo_capital.loc[((solo_capital.lon <= -58) & (solo_capital.lon >= -59)),:]

## Subtes

In [4]:
subtes = pd.read_csv('datos/estaciones-de-subte.csv', sep = ",")

In [ ]:
subtes.rename(columns = {'X': 'lon', 'Y': 'lat'}, inplace = True)

In [ ]:
solo_capital.loc[:,'distancia_subtes'] = solo_capital.loc[:,['lat','lon']].apply(lambda x: distancia_minima(x, subtes), axis = 1)

## Monumentos

In [6]:
monumentos = pd.read_csv('datos/monumentos.csv', sep = ",")

In [9]:
solo_capital.loc[:,'distancia_monumentos'] = solo_capital.loc[:,['lat','lon']].apply(lambda x: distancia_minima(x, monumentos), axis = 1)

## Ferrocarriles

In [8]:
ferrocarriles = pd.read_csv('datos/estaciones-de-ferrocarril.csv', sep = ";")

In [14]:
ferrocarriles.rename(columns = {'LAT': 'lat', 'LNG': 'lon' }, inplace = True)

In [ ]:
solo_capital.loc[:,'distancia_ferrocarril'] = solo_capital.loc[:,['lat','lon']].apply(lambda x: distancia_minima(x, ferrocarriles), axis = 1)

## Estadios

In [10]:
estadios = pd.read_csv('datos/estadios.csv', sep = ";")

In [14]:
estadios.rename(columns = {'LAT': 'lat', 'LONG': 'lon' }, inplace = True)

In [2]:
solo_capital.loc[:,'distancia_estadios'] = solo_capital.loc[:,['lat','lon']].apply(lambda x: distancia_minima(x, estadios), axis = 1)

# Zona de inundaciones

In [2]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, Point
from shapely import wkt

In [3]:
inundaciones = gpd.read_file('datos/mapas/inundaciones.dbf')
inundaciones.head()

,provincia,geometry
0,Buenos Aires,POLYGON ((-58.80922221534311 -34.5343527692759...
1,Buenos Aires,POLYGON ((-58.55217881951273 -34.7005448538739...
2,Buenos Aires,"POLYGON ((-57.99419000000001 -34.957731, -57.9..."
3,Buenos Aires,POLYGON ((-58.35363374172896 -34.7235757806278...
4,Buenos Aires,POLYGON ((-58.18616301265681 -34.7955152796631...


In [20]:
solo_capital.loc[:,'Zona_inundacion'] = solo_capital.loc[:,['lat','lon']].apply\
(lambda x: inundaciones.contains(Point(x[1], x[0])).any(), axis = 1)

# Cercania a las villas

In [11]:
villas = gpd.read_file('datos/villas/villas.dbf')
villas.head()

,provincia,geometry
0,Buenos Aires,POLYGON ((-58.80922221534311 -34.5343527692759...
1,Buenos Aires,POLYGON ((-58.55217881951273 -34.7005448538739...
2,Buenos Aires,"POLYGON ((-57.99419000000001 -34.957731, -57.9..."
3,Buenos Aires,POLYGON ((-58.35363374172896 -34.7235757806278...
4,Buenos Aires,POLYGON ((-58.18616301265681 -34.7955152796631...


In [17]:
aux = villas.loc[villas.provincia.str.contains('Ciudad'),:]

In [24]:
solo_capital.loc[:,'distancia_villas'] = solo_capital.loc[:,['lat','lon']].apply\
(lambda x: min(aux.distance(Point(x[1], x[0]))) * 100, axis = 1)

In [30]:
min(solo_capital.distancia_villas)

0.0

In [ ]:
# Algunos dan negativo porque la distancia era menor que el radio y yo se los reste.
# De ultima lo cambio despues recorro y si es negativo lo reemplazo por cero

## Exportacion del archivo

In [27]:
#solo_capital.to_csv('solo_capital.csv', index = False)
solo_capital.to_csv('/home/agustin/Escritorio/solo_capital2.csv', index = False)